# VideoNetClassification

Collaborators:

- Yahia Ehab
- Mariam Amr
- Mohamed Khaled

## Installation

In [1]:
# !pip install -q imageio
# !pip install -q opencv-python
# !pip install -q git+https://github.com/tensorflow/docs

## Imports

In [1]:
# @title Import the necessary modules
# TensorFlow and TF-Hub modules.
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed

logging.set_verbosity(logging.ERROR)

# Some modules to help with reading the UCF101 dataset.
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np

# Some modules to display an animation using imageio.
import imageio
from IPython import display

from urllib import request  # requires python3

## Data Loading

In [24]:
import pandas as pd

# Helper functions for the UCF101 dataset
UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = None
_CACHE_DIR = tempfile.mkdtemp()
unverified_context = ssl._create_unverified_context()

def list_ucf_videos():
    """Lists videos available in UCF101 dataset."""
    global _VIDEO_LIST
    if not _VIDEO_LIST:
        index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode("utf-8")
        videos = re.findall("(v_[\w_]+\.avi)", index)
        _VIDEO_LIST = sorted(set(videos))
    return list(_VIDEO_LIST)

def fetch_ucf_video(video):
    """Fetches a video and cache into local filesystem."""
    cache_path = os.path.join(_CACHE_DIR, video)
    if not os.path.exists(cache_path):
        urlpath = request.urljoin(UCF_ROOT, video)
        print("Fetching %s => %s" % (urlpath, cache_path))
        data = request.urlopen(urlpath, context=unverified_context).read()
        open(cache_path, "wb").write(data)
    return cache_path

def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(224, 224)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if max_frames != 0 and len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames) / 255.0

def to_gif(images):
    converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
    imageio.mimsave('./animation.gif', converted_images, duration=40)
    return embed.embed_file('./animation.gif')

# Define a function to create DataFrame with video paths and labels
def create_dataframe(num_videos=300):
    video_paths = []
    labels = []

    # List all UCF101 videos
    ucf_videos = list_ucf_videos()

    # Randomly select videos
    random_videos = random.sample(ucf_videos, num_videos)

    # Extract labels from video filenames
    for video in random_videos:
        label = video.split('_')[1]
        video_paths.append(fetch_ucf_video(video))
        labels.append(label)

    # Create DataFrame
    df = pd.DataFrame({'video_paths': video_paths, 'labels': labels})
    return df

# Create DataFrame with video paths and labels
df = create_dataframe()

# # Display the DataFrame
print(df.head())


Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_StillRings_g24_c01.avi => C:\Users\EL-WAT~1\AppData\Local\Temp\tmp_roddtfq\v_StillRings_g24_c01.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_JugglingBalls_g22_c03.avi => C:\Users\EL-WAT~1\AppData\Local\Temp\tmp_roddtfq\v_JugglingBalls_g22_c03.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_SalsaSpin_g16_c01.avi => C:\Users\EL-WAT~1\AppData\Local\Temp\tmp_roddtfq\v_SalsaSpin_g16_c01.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_BreastStroke_g23_c03.avi => C:\Users\EL-WAT~1\AppData\Local\Temp\tmp_roddtfq\v_BreastStroke_g23_c03.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_VolleyballSpiking_g11_c06.avi => C:\Users\EL-WAT~1\AppData\Local\Temp\tmp_roddtfq\v_VolleyballSpiking_g11_c06.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_Kayaking_g11_c05.avi => C:\Users\EL-WAT~1\AppData\Local\Temp\tmp_roddtfq\v_Kayaking_g11_c05.avi
Fetching https://www.crcv.uc

In [25]:
# Save the DataFrame to a CSV file
df.to_csv('ucf101_videos_labels.csv', index=False)

In [26]:
df.head()

,video_paths,labels
0,C:\Users\EL-WAT~1\AppData\Local\Temp\tmp_roddt...,StillRings
1,C:\Users\EL-WAT~1\AppData\Local\Temp\tmp_roddt...,JugglingBalls
2,C:\Users\EL-WAT~1\AppData\Local\Temp\tmp_roddt...,SalsaSpin
3,C:\Users\EL-WAT~1\AppData\Local\Temp\tmp_roddt...,BreastStroke
4,C:\Users\EL-WAT~1\AppData\Local\Temp\tmp_roddt...,VolleyballSpiking


In [27]:
df['video_paths'][0]

'C:\\Users\\EL-WAT~1\\AppData\\Local\\Temp\\tmp_roddtfq\\v_StillRings_g24_c01.avi'

### Load Video as GIF 

Create `/GIFs` dir

In [33]:
from tensorflow.keras.applications.inception_v3 import preprocess_input

def to_gif2(images,index): 
    # resized_images = np.zeros((len(images), 299, 299, 3), dtype=np.uint8)
    # for i, frame in enumerate(images):
    #     resized_images[i] = resize(frame, (299, 299), anti_aliasing=True) * 255
    converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
    imageio.mimsave('./GIFs/animation'+index+'.gif', converted_images, duration=40)

# #resize each image to 299x299 (as is prefered for inceptionv3) and save as gif

# for i in range(0, 300):
#     # Load the first video from the DataFrame
#     video_path = df['video_paths'][i]
#     video = load_video(video_path)
#     to_gif2(video,str(i))
def preprocess_frame(frame):
    preprocessed_frame = preprocess_input(frame)
    return preprocessed_frame

frames = [] # a '2d' array where each element is a group of frames corresponding to one video
for i in range(0, 300):
    # Load the first video from the DataFrame
    video_path = df['video_paths'][i]
    video = load_video(video_path, resize=(299, 299))
    converted_video = np.clip(video*255, 0, 255).astype(np.uint8)
    frames.append([preprocess_frame(frame) for frame in converted_video])
    print("finished video: ", i)

# extract labels from the dataframe
labels = df['labels'].values


MemoryError: Unable to allocate 460. MiB for an array with shape (225, 299, 299, 3) and data type float64

In [30]:
print(frames[0])

[[[[ 90  84  50]
   [ 92  86  52]
   [ 93  87  53]
   ...
   [ 29  28  31]
   [ 52  52  52]
   [ 99  99  99]]

  [[ 89  83  49]
   [ 90  84  50]
   [ 92  86  52]
   ...
   [ 25  24  27]
   [ 44  44  45]
   [ 84  84  84]]

  [[ 88  82  47]
   [ 89  83  49]
   [ 90  84  50]
   ...
   [ 22  21  24]
   [ 34  34  34]
   [ 62  62  62]]

  ...

  [[ 26  48  62]
   [ 34  56  71]
   [ 38  68  92]
   ...
   [  9   9   4]
   [  9   9   4]
   [ 10  10   5]]

  [[ 19  43  46]
   [ 27  52  56]
   [ 32  64  75]
   ...
   [  8   8   3]
   [  8   8   3]
   [  8   8   3]]

  [[  1  16  18]
   [  1  23  26]
   [  2  34  44]
   ...
   [  6   6   1]
   [  6   6   1]
   [  5   5   0]]]


 [[[ 95  87  53]
   [ 96  88  54]
   [ 97  89  55]
   ...
   [ 62  61  66]
   [ 96  95  99]
   [ 63  62  65]]

  [[ 95  86  53]
   [ 96  88  54]
   [ 97  89  55]
   ...
   [ 52  51  56]
   [ 90  89  92]
   [ 77  76  79]]

  [[ 92  84  50]
   [ 94  87  52]
   [ 96  88  54]
   ...
   [ 36  35  40]
   [ 72  71  74]
   [ 95  94

## Preprocessing

- CNN (InceptionV3 Model)
    1. Image Size should be 299*299

- RNN
    1. LTSM

### CNN

In [5]:
# Load the InceptionV3 model from TensorFlow Hub
feature_extractor = hub.KerasLayer(
    "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4", trainable=False
)

In [6]:
def load_gifs(gif_dir):
    gif_paths = [
        os.path.join(gif_dir, file)
        for file in os.listdir(gif_dir)
        if file.endswith(".gif")
    ]
    return gif_paths


def load_gif(gif_path):
    gif = imageio.mimread(gif_path)
    return [frame.astype(np.float32) / 255.0 for frame in gif]


# Function to extract features using InceptionV3
def extract_video_features(video_frames):
    video_features = []
    for frame in video_frames:
        frame = tf.expand_dims(
            frame, axis=0
        )  # Resize the frame to (1, height, width, channels)
        frame_features = feature_extractor(frame)
        video_features.append(frame_features)
    video_features = np.array(video_features)
    # Aggregate features over time (e.g., taking the average)
    video_features = np.mean(video_features, axis=0)
    return video_features


gif_dir = "./GIFs"
gif_paths = load_gifs(gif_dir)

for gif_path in gif_paths:
    gif_frames = load_gif(gif_path)
    gif_features = extract_video_features(gif_frames)